# Purpose

Purpose:  
For a table or spreadsheet with dates, match up the next date so you can calculate the number of days between occurrances. This will also allow group by's so you can constrain the set of intervals by any keys in the data.  
Input:  
It rquires either 'order_date_interval_test.csv' or 'order_date_interval_dev.csv' from the git repo to be downloaded in the same directory from which you are running this notebook.  
Output:  
It will output a .csv file with the original data, the next date in a new column and the day intervals as an integer value in a new column called "intervals.csv". It then creates statistics along any combination of keys like, min , max, count, and mean called keystat.csv.  
Example:  
lets say you had a file of all the purchases of all products your customers bought for the last 3 months. How often (mean number of days between each purchase) does customer 123 buy product 989 over the past 3 months? Is there a pattern that a simple average would hide such as once a week then once every 3 weeks, then once a week. Developing scalar time features is relatively straight forward once you put all the pieces together.

In [2]:
import csv
import os
import pandas as pd
from datetime import datetime
import numpy as np

This assumes you are running your notebook in the same directories as the order_date_interval dev and test files.
"raw_orders_df" is the name of the panda data frame we are creating
"pd.read_csv" is the actual panda function that:
1) reads the csv in this case 'order_date_interval_test.csv'
2) "names" inserts all the column names (it assumes to csv header columns)
3) all the other stuff converts the dates in the spreadsheet to a "datetime" data type

"print(Raw file Shape" tells you how many rows and clolumns you have-- for test it is Raw file Shape: (10826, 5)
"display(raw_orders_df.info())" tells you about the columns data type. Its critical Order Date is datetime.
It also shows you the memory usage at the bottom.

In [12]:
raw_orders_df = pd.read_csv('order_date_interval_test.csv', 
                            names = ['RowID', 'CustId',  'ProdId', 'Cases','Order_Date'],
                            parse_dates = ['Order_Date'],
                            infer_datetime_format = True,
                           date_parser = pd.to_datetime,
                           )

print("Raw file Shape: {}\n".format(raw_orders_df.shape))
display(raw_orders_df.info())

Raw file Shape: (10826, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10826 entries, 0 to 10825
Data columns (total 5 columns):
RowID         10826 non-null int64
CustId        10826 non-null int64
ProdId        10826 non-null int64
Cases         10826 non-null int64
Order_Date    10826 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 423.0 KB


None

In [13]:
raw_orders_df.head()

,RowID,CustId,ProdId,Cases,Order_Date
0,8664,4,6,0,2018-05-31
1,1,4,6,0,2018-06-01
2,49402,4,6,1,2018-06-05
3,58729,4,6,0,2018-06-07
4,108743,4,6,0,2018-06-14


Sort and Scrub data:
We want all the unique CustId and ProdId rows to be sorted in time (order_date)in ascending order
The files are already sorted, but  in the case of a file you are modifying it has to be in the right order for everything else to work.
In the next cell we are going to remove all the order lines where the customer did not order any cases. We will create a new dataframe called orders_df since it will be set up in a way we can begin time series operations.  

In [14]:
raw_orders_df.sort_values(by=['CustId','ProdId','Order_Date'])

,RowID,CustId,ProdId,Cases,Order_Date
0,8664,4,6,0,2018-05-31
1,1,4,6,0,2018-06-01
2,49402,4,6,1,2018-06-05
3,58729,4,6,0,2018-06-07
4,108743,4,6,0,2018-06-14
5,152220,4,6,1,2018-06-19
6,162016,4,6,0,2018-06-21
7,207346,4,6,0,2018-06-26
8,217613,4,6,0,2018-06-28
9,261836,4,6,1,2018-07-03


This is where we qualify the unique set of events, in this case (excuse the pun) only the rows where we ordered some cases of something. Note the .copy() function prevents subsequent "Setting with Copy warnings"

In [15]:
orders_df = raw_orders_df.loc[raw_orders_df.Cases !=0].copy()

Check to make sure we only got the rows we want to compare.

In [16]:
orders_df.head()

,RowID,CustId,ProdId,Cases,Order_Date
2,49402,4,6,1,2018-06-05
5,152220,4,6,1,2018-06-19
9,261836,4,6,1,2018-07-03
11,317943,4,6,1,2018-07-10
13,373136,4,6,1,2018-07-17


#  shift() or (-1)  
The shift command is how we going to populate the new next_date column with the prior order date if and only if the custid and prodid is the same. That is why we shiftin the context of a groupby function. The shift() moves something down 1 row where as shift(-1) moves it up 1 row

In [17]:
orders_df['next_date'] = orders_df.groupby(['CustId','ProdId'])['Order_Date'].shift(-1)

In [19]:
orders_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4404 entries, 2 to 10823
Data columns (total 6 columns):
RowID         4404 non-null int64
CustId        4404 non-null int64
ProdId        4404 non-null int64
Cases         4404 non-null int64
Order_Date    4404 non-null datetime64[ns]
next_date     3901 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(4)
memory usage: 240.8 KB


In [31]:
orders_df.head(15)


,RowID,CustId,ProdId,Cases,Order_Date,next_date
2,49402,4,6,1,2018-06-05,2018-06-19
5,152220,4,6,1,2018-06-19,2018-07-03
9,261836,4,6,1,2018-07-03,2018-07-10
11,317943,4,6,1,2018-07-10,2018-07-17
13,373136,4,6,1,2018-07-17,2018-07-24
15,430542,4,6,1,2018-07-24,2018-07-26
16,441411,4,6,2,2018-07-26,2018-08-16
27,152221,4,7,1,2018-06-19,2018-07-03
31,261837,4,7,2,2018-07-03,2018-07-17
35,373137,4,7,1,2018-07-17,2018-07-24


In [22]:
orders_df = orders_df.dropna()

In [23]:
order_w_cases = orders_df.copy()

In [24]:
order_w_cases.head()

,RowID,CustId,ProdId,Cases,Order_Date,next_date
2,49402,4,6,1,2018-06-05,2018-06-19
5,152220,4,6,1,2018-06-19,2018-07-03
9,261836,4,6,1,2018-07-03,2018-07-10
11,317943,4,6,1,2018-07-10,2018-07-17
13,373136,4,6,1,2018-07-17,2018-07-24


# date orders  
Now that we have the dates structured in our tables, we want to find the difference between the current date and the next date. We have a null for the next date of the last row in the group so, we will get rid of those with the dropna in the next cell. Finally, we will convert Day_Interval, in the timedelta data type of "x Days" to a regular integer value by using the date time days function (dt.days).

In [32]:
order_w_cases['Day_Interval'] = (order_w_cases.next_date - order_w_cases.Order_Date)

In [33]:
order_w_cases.dropna()

,RowID,CustId,ProdId,Cases,Order_Date,next_date,Day_Interval
2,49402,4,6,1,2018-06-05,2018-06-19,14 days
5,152220,4,6,1,2018-06-19,2018-07-03,14 days
9,261836,4,6,1,2018-07-03,2018-07-10,7 days
11,317943,4,6,1,2018-07-10,2018-07-17,7 days
13,373136,4,6,1,2018-07-17,2018-07-24,7 days
15,430542,4,6,1,2018-07-24,2018-07-26,2 days
16,441411,4,6,2,2018-07-26,2018-08-16,21 days
27,152221,4,7,1,2018-06-19,2018-07-03,14 days
31,261837,4,7,2,2018-07-03,2018-07-17,14 days
35,373137,4,7,1,2018-07-17,2018-07-24,7 days


In [34]:
order_w_cases.Day_Interval = order_w_cases.Day_Interval.dt.days

In [35]:
order_w_cases.head()

,RowID,CustId,ProdId,Cases,Order_Date,next_date,Day_Interval
2,49402,4,6,1,2018-06-05,2018-06-19,14
5,152220,4,6,1,2018-06-19,2018-07-03,14
9,261836,4,6,1,2018-07-03,2018-07-10,7
11,317943,4,6,1,2018-07-10,2018-07-17,7
13,373136,4,6,1,2018-07-17,2018-07-24,7


In a regular pipeline , data comes from a database and needs to be tested to make sure you have in fact accounted for each row. Since RowID was the primary key in the database I could join the output to the original database. This is also good data for other feature construction such as day of the week analysis or weather correlation. We are going to change the insex to rowID and then export it to the interval.csv file

In [51]:
order_w_cases.set_index('RowID', inplace = True)

In [52]:
order_w_cases.head()

,CustId,ProdId,Cases,Order_Date,next_date,Day_Interval
RowID,,,,,,
49402,4,6,1,2018-06-05,2018-06-19,14
152220,4,6,1,2018-06-19,2018-07-03,14
261836,4,6,1,2018-07-03,2018-07-10,7
317943,4,6,1,2018-07-10,2018-07-17,7
373136,4,6,1,2018-07-17,2018-07-24,7


In [36]:
order_w_cases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3901 entries, 2 to 10819
Data columns (total 7 columns):
RowID           3901 non-null int64
CustId          3901 non-null int64
ProdId          3901 non-null int64
Cases           3901 non-null int64
Order_Date      3901 non-null datetime64[ns]
next_date       3901 non-null datetime64[ns]
Day_Interval    3901 non-null int64
dtypes: datetime64[ns](2), int64(5)
memory usage: 243.8 KB


In [54]:
order_w_cases.to_csv('interval.csv')

now we work on all the summary stats based on the intervals

In [37]:
keystat = order_w_cases.groupby(['CustId', 'ProdId'], as_index=False).agg({'Day_Interval': [min, max, 'count','mean'], 'Cases':[min,max,sum,'mean']})

In [38]:
keystat.head()

CustId ProdId Day_Interval                      Cases                   
                         min max count       mean   min max sum       mean
0      4      6            2  21     7  10.285714     1   2   8   1.142857
1      4      7            2  21     5  11.600000     1   2   6   1.200000
2      4     18            5  16     5   9.800000     1   2   7   1.400000
3      4     22            7  19     7  10.000000    10  20  82  11.714286
4      4     23            2  14    12   6.000000     4  13  79   6.583333

Wow - that is pretty ugly lets merge the upper header row with the lower header row

In [39]:
keystat.columns = ["_".join(x) for x in keystat.columns.ravel()]

In [39]:
keystat.head()

,CustId_,ProdId_,Day_Interval_min,Day_Interval_max,Day_Interval_count,Day_Interval_mean,Cases_min,Cases_max,Cases_sum,Cases_mean
0,4,6,2,21,7,10.285714,1,2,8,1.142857
1,4,7,2,21,5,11.600000,1,2,6,1.200000
2,4,18,5,16,5,9.800000,1,2,7,1.400000
3,4,22,7,19,7,10.000000,10,20,82,11.714286
4,4,23,2,14,12,6.000000,4,13,79,6.583333


In [40]:
keystat.rename(columns ={'CustId_':'CustId', 'ProdId_':'ProdId'}, inplace =True)

In [41]:
keystat=keystat.copy()

In [42]:
keystat['Day_Interval_mean'] = keystat['Day_Interval_mean'].astype(int)

Here we have a classic problem. The "Dev" data set produces a "cases_mean" that is integer and the "Test" dataset produces a much more realistic float64 data type so, when you switch files you are going to get an error if you run dev.
If you are running dev don't run the next row , just skip it.

In [46]:
keystat['Cases_mean'] = keystat['Cases_mean'].astype(int)

In [47]:
keystat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503 entries, 0 to 502
Data columns (total 10 columns):
CustId                503 non-null int64
ProdId                503 non-null int64
Day_Interval_min      503 non-null int64
Day_Interval_max      503 non-null int64
Day_Interval_count    503 non-null int64
Day_Interval_mean     503 non-null int64
Cases_min             503 non-null int64
Cases_max             503 non-null int64
Cases_sum             503 non-null int64
Cases_mean            503 non-null int64
dtypes: int64(10)
memory usage: 43.2 KB


In [48]:
keystat.head()

,CustId,ProdId,Day_Interval_min,Day_Interval_max,Day_Interval_count,Day_Interval_mean,Cases_min,Cases_max,Cases_sum,Cases_mean
0,4,6,2,21,7,10,1,2,8,1
1,4,7,2,21,5,11,1,2,6,1
2,4,18,5,16,5,9,1,2,7,1
3,4,22,7,19,7,10,10,20,82,11
4,4,23,2,14,12,6,4,13,79,6


In [48]:
keystat.to_csv('keystat.csv')